In [1]:
# 使用OpenCV3.x-Python检测AKAZE特征点
 
import cv2
import numpy
 
def main():
    img = cv2.imread("lena.png")
    # 检测
    akaze = cv2.AKAZE_create()
    keypoints = akaze.detect(img, None)
    
    # 显示
    # 必须要先初始化img2
    img2 = img.copy()
    img2 = cv2.drawKeypoints(img, keypoints, img2, color=(0,255,0))
    cv2.imshow('Detected AKAZE keypoints', img2)
    cv2.waitKey(0)
    
if __name__ == '__main__':
    main()

In [2]:
# from https://blog.csdn.net/sinat_36811967/article/details/84136358
import cv2

# 检测两张图片里的所有特征点：
img1 = cv2.imread("WIN_20201008_13_00_24_Pro.jpg")
img2 = cv2.imread("WIN_20201008_13_00_26_Pro.jpg")

akaze = cv2.AKAZE_create()

kp1,des1 = akaze.detectAndCompute(img1,None)
kp2,des2 = akaze.detectAndCompute(img2,None)

# 用Brute Force Match 匹配：选取几个最近的，（这里选2个）
bf = cv2.BFMatcher(cv2.NORM_L2)
matches = bf.knnMatch(des1,des2,k=2)

# 优化就是限定一个条件，如第二近的点距离要大于最近点距离的两倍才算好的match；
goodMatch = []
for m,n in matches:
    if m.distance < 0.50 * n.distance:
        goodMatch.append(m)
        
# 最后是连线可视化
res = cv2.drawMatches(img1, kp1, img2, kp2, goodMatch[:], None, flags=2)
cv2.imshow('res',res)
cv2.waitKey(0)
cv2.destroyAllWindows()

## DMatch类
https://www.bbsmax.com/A/obzbr0x0zE/

1、2、3不用说，是三个构造函数。接着, 

int queryIdx –>是**测试图像**(图像2)的特征点描述符（descriptor）的下标，同时也是描述符对应特征点（keypoint)的下标。

int trainIdx –> 是**样本图像**(图像1)的特征点描述符的下标，同样也是相应的特征点的下标。

int imgIdx –>当样本是多张图像的话有用。

float distance –>代表这一对匹配的特征点描述符（本质是向量）的**欧氏距离**，数值越小也就说明两个特征点越相像。

最后， 也就是一个小于操作符的重载，用于比较和排序。 比较的是上述的distance，当然是越小越好。

In [3]:
# AKAZE特征点检测和匹配
def GetImageMatches(img1,img2):
    akaze = cv2.AKAZE_create()
    kp1, desc1 = akaze.detectAndCompute(img1,None)#检测特征点并返回特征点和特征点描述符
    kp2, desc2 = akaze.detectAndCompute(img2,None)
    
    matcher = cv2.BFMatcher(crossCheck = True)#True: 两个特征点之间距离须互为最近
    matches = matcher.match(desc1, desc2)#返回最佳匹配特征点对和之间距离(DMatch类)，共141个点对
    
    matches = sorted(matches, key = lambda x:x.distance)#按照距离将特征点对升序排列
    #(float) distance 代表这一对匹配的特征点描述符（本质为向量）的欧式距离，数值越小说明两个特征点越像
    
    return kp1,desc1,kp2,desc2,matches

In [4]:
# 得到2个图像中匹配点的坐标
def GetAlignedMatches(kp1,desc1,kp2,desc2,matches):
    # 防止万一matches 列表还没排序：
    matches = sorted(matches, key = lambda x:x.distance) #内建函数sorted返回一个新list
    
    # 从matches中取回两个图像的keypoints的对应的序号
    img1idx = np.array([m.queryIdx for m in matches]) #图像2的特征点下标，将列表转换成多维数组（方便运算）
    img2idx = np.array([m.trainIdx for m in matches]) #图像1的特征点下标
    
    # 过滤掉不匹配的keypoints（matches中元素数量 > 各自点集元素数量？）
    kp1_ = (np.array(kp1))[img1idx]
    kp2_ = (np.array(kp2))[img2idx]
    
    # 取回匹配的keypoints的图像坐标
    img1pts = np.array([kp.pt for kp in kp1_])
    img2pts = np.array([kp.pt for kp in kp2_])
    
    return img1pts,img2pts,img1idx,img2idx

In [ ]:
#
def EstimateFundamentalMatrix(img1pts,img2pts):
    #变成齐次坐标
    if img1pts.shape[1] == 2:
        img1pts = cv2.convertPointsToHomogeneous(img1pts)[:,0,:]

In [ ]:
# 
def GetTriangulatedPts(img1pts,img2pts,K,R,t,triangulateFunc)

In [5]:
import numpy as np
import cv2

#Reading two images for reference
img1 = cv2.imread("WIN_20201008_13_00_24_Pro.jpg")#检测到258个特征点
img2 = cv2.imread("WIN_20201008_13_00_26_Pro.jpg")#检测到284个特征点

#Converting from BGR to RGB format
img1 = img1[:,:,::-1]
img2 = img2[:,:,::-1]

# AKAZE特征点检测和匹配
kp1,desc1,kp2,desc2,matches = GetImageMatches(img1,img2)
# 对齐两个关键点向量
img1pts,img2pts,img1idx,img2idx = GetAlignedMatches(kp1,desc1,kp2,desc2,matches)

# 计算基本矩阵——8点算法
img1pts_, img2pts_ = img1pts[:8],img2pts[:8]
Fgt, mask = cv2.findFundamentalMat(img1pts_,img2pts_,method=cv2.FM_8POINT)
F = sf

ValueError: too many values to unpack (expected 2)